In [1]:
import time
import N38_scraper
from N38_scraper import get_allnews_urls
from N38_scraper import parse_page

urls = get_allnews_urls(begin_page=1, end_page=3, verbose=True)
print('%d urls for all news' % len(urls))

get briefing statement urls 1 / 3
get briefing statement urls 2 / 3
get briefing statement urls 3 / 3
12 urls for all news


In [2]:
def pprint(json_object):
    for k, v in json_object.items():
        print('{} : {} ..'.format(k, str(v)[:100]))
    print('\n')

SLEEP = 0.5

for url in urls[:3]:
    json_object = parse_page(url)
    pprint(json_object)

title : Reports of “Secret” North Korean Missile Bases: Much Ado about Not Much ..
time : January 28, 2019 ..
content : President Trump and North Korean leader Kim Jong Un have now agreed to meet for their second summit  ..
url : https://www.38north.org/2019/01/ddepetris012819/ ..
scrap_time : 2019-01-30-04 05:05:21 ..


title : Inspector O and the New Cat ..
time : January 23, 2019 ..
content : I did not know if Inspector O would show up, and so was greatly relieved when I spotted him edging a ..
url : https://www.38north.org/2019/01/jchurch012319/ ..
scrap_time : 2019-01-30-04 05:05:22 ..


title : Why is Kim Yong Chol Still Pyongyang’s Choice for US-DPRK Relations? ..
time : January 18, 2019 ..
content : Kim Yong Chol has arrived in Washington to meet with Secretary of State Pompeo and President Trump.  ..
url : https://www.38north.org/2019/01/svogler011819/ ..
scrap_time : 2019-01-30-04 05:05:23 ..




In [3]:
from N38_scraper import yield_latest_allnews

begin_date = '2018-07-01'
max_num = 50
sleep = 1.0

for i, json_obj in enumerate(yield_latest_allnews(begin_date, max_num, sleep)):
    title = json_obj['title']
    time = json_obj['time']
    print('[{} / {}] ({}) {}'.format(i+1, max_num, time, title))

[1 / 50] (January 28, 2019) Reports of “Secret” North Korean Missile Bases: Much Ado about Not Much
[2 / 50] (January 23, 2019) Inspector O and the New Cat
[3 / 50] (January 18, 2019) Why is Kim Yong Chol Still Pyongyang’s Choice for US-DPRK Relations?
[4 / 50] (January 18, 2019) Advancing Peace and Security on the Korean Peninsula: The Case for Koreanization
[5 / 50] (January 28, 2019) Reports of “Secret” North Korean Missile Bases: Much Ado about Not Much
[6 / 50] (January 23, 2019) Inspector O and the New Cat
[7 / 50] (January 18, 2019) Why is Kim Yong Chol Still Pyongyang’s Choice for US-DPRK Relations?
[8 / 50] (January 18, 2019) Advancing Peace and Security on the Korean Peninsula: The Case for Koreanization
[9 / 50] (January 16, 2019) Beach Watch: North Korea’s Wonsan Gets a Boost
[10 / 50] (January 9, 2019) North Korea’s Yongbyon Nuclear Facilities: Well Maintained but Showing Limited Operations
[11 / 50] (January 9, 2019) Kim Jong Un’s New Year’s Speech: On the Domestic Front


[92 / 50] (July 16, 2018) How the North Korean Economy Should—and Shouldn’t—be Used in Negotiations
[93 / 50] (July 12, 2018) Putin, Trump and North Korea: The Road to Washington Leads Through Asia
[94 / 50] (July 10, 2018) The Singapore Summit in Perspective: Lessons from the Israeli-Arab Conflict
[95 / 50] (July 9, 2018) Lessons from the Unhappy History of Verification in North Korea
[96 / 50] (July 9, 2018) What the North Koreans Really Said on July 7
[97 / 50] (July 6, 2018) North Korea’s Yongbyon Nuclear Research Center: Testing of Reactor Cooling Systems; Construction of Two New Non-Industrial Buildings
[98 / 50] (July 6, 2018) Building Trust on the Korean Peninsula: The Role of CBMs in Nuclear Negotiations
[99 / 50] (July 3, 2018) 38 North Special Report: Recent Changes in Kim Jong Un’s High Command
Stop scrapping. 0 / 50 news was scrapped
The oldest news has been created after 2018-07-01
